<a href="https://colab.research.google.com/github/ENRICOROBLESDELRIO/data_boot_camp_final_project/blob/lfba_branch/Data_Preprocessing_Company_Table_V02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Libraries

In [ ]:
# Libraries and dependencies

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import math
from sklearn.metrics import accuracy_score
from google.colab import files
from google.colab import drive
import psycopg2


# Import for models
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# Google Cloud SQL connexion:
# Code taken from: https://stackoverflow.com/questions/53180140/how-to-access-cloud-sql-from-google-colab
!gcloud auth login
!gcloud config set project endeavor-intelligence-unit
# !gcloud sql instances describe finalprojectdb

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=WXWsNcrqyVJJ3XW0VoonjDxEXo1PlP&prompt=consent&access_type=offline&code_challenge=wgZsU1e_YyzW7nGQSGad9VbfXK717MWwY15SHXowZe0&code_challenge_method=S256

Enter verification code: 4/1AX4XfWjSeRJ6gwOi6FtZx9QSQ1dsgg7T8NVwy9gzXHAkU69Y2nHMvHnxhQU
ERROR: gcloud crashed (InvalidGrantError): (invalid_grant) Bad Request

If you would like to report this issue, please run the following command:
  gcloud feedback

To check gcloud for common problems, please run the following command:
  gcloud info --ru

In [ ]:

# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy

# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="endeavor-intelligence-unit:northamerica-northeast2:finalprojectdb"=tcp:5432 &
!sleep 30s

--2021-11-14 02:20:38--  https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64
Resolving dl.google.com (dl.google.com)... 108.177.12.93, 108.177.12.91, 108.177.12.190, ...
Connecting to dl.google.com (dl.google.com)|108.177.12.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16638816 (16M) [application/octet-stream]
Saving to: ‘cloud_sql_proxy’

cloud_sql_proxy     100%[===================>]  15.87M  --.-KB/s    in 0.08s   

2021-11-14 02:20:38 (212 MB/s) - ‘cloud_sql_proxy’ saved [16638816/16638816]

nohup: appending output to 'nohup.out'


In [ ]:
# Connection to protect access and users to the database using a drive file: 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Connection to the file with user and access to the database: 
# Each user should use its own personal path to get access to the database and switch it below in open() function. 

with open('/content/drive/MyDrive/Personal/Data_Analytics_Boot_Camp/Final_project/Second_Deliverable/key_config.txt', "r") as file:
    FileasList = file.readlines()
    username = FileasList[0]
    password = FileasList[1]

    new_line = []
    # Loop through the file
    for i in FileasList:
        # Get rid of any new lines ("enter")
        i = i.replace("\n", "")
        # Add the cleaned data to the new_line list
        new_line.append(i)
    # Set the username to be the first object
    username = new_line[0]
    # Set the password to be the second object
    password = new_line[1]
    # Set the host to be the third object
    host = new_line[2]
    # Set the port to be the four object
    port = new_line[3]
    # Set the database to be the five object
    database = new_line[4]

# Check of Companies

In [ ]:
# Open Conection

# Connexion throught psycopg2 Google Cloud SQL data base
conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

In [ ]:
# Query to connect with SQL data base
mySQL_Query = pd.read_sql_query('''select "id_company" FROM "Rounds" GROUP BY "id_company"''', conn)
my_Rounds = pd.DataFrame(mySQL_Query)
my_Rounds.set_index("id_company",inplace=True)
companies_in_rounds=len(my_Rounds)
companies_in_rounds

560

In [ ]:
# Query to connect with SQL data base
mySQL_Query = pd.read_sql_query('''select "id_company" FROM "Founders" GROUP BY "id_company"''', conn)
my_Founders = pd.DataFrame(mySQL_Query)
my_Founders.set_index("id_company",inplace=True)
companies_in_founders = len(my_Founders)
companies_in_founders

512

In [ ]:
companies_in_rounds_and_founders = len(my_Rounds.join(my_Founders,how='inner'))
companies_in_rounds_and_founders

510

In [ ]:
# Close Connection
conn.close()

# Rounds Table Data Cleaning

In [ ]:
# Open Conection

# Connexion throught psycopg2 Google Cloud SQL data base
conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

In [ ]:
# Query to connect with SQL data base
SQL_Query = pd.read_sql_query('''select * FROM "Rounds" ''', conn)
original_rounds_df = pd.DataFrame(SQL_Query)

In [ ]:
# Close Connection
conn.close()

In [ ]:
# Dataframe length
original_rounds_len=len(original_rounds_df)
original_rounds_len

1236

In [ ]:
# Data base of VC Rounds in entrepreneurs 
original_rounds_df.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team
0,3,E-Commerce,Seed,2017,33.0,8,3,1,0,1,2017,1400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,SaaS,Seed,2018,10.0,0,2,0,0,1,2018,10500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,SaaS,Angel,2015,11.0,14,1,1,0,1,2015,200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,IT services,Seed,2016,10.0,0,1,0,0,1,2016,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23,E-Commerce,Seed,2013,1001.0,0,1,1,1,0,2013,150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Count NaN
original_rounds_df.isna().sum()

id_company                   0
category                     0
deal_type                    0
foundation_year              0
number_employees             7
increase_employees           0
number_investors             0
current_status               0
foreign_vc                   0
local_vc                     0
round_year                   0
round_amount                 0
support_company              5
manufacture_equipment        5
develop_new_tech             5
grow_company                 5
add_new_services             5
develop_business             5
financial_banking            5
develop_products_services    5
marketing_strategies         5
expand_internationality      5
consolidate_company          5
increase_team                5
dtype: int64

In [ ]:
original_rounds_df['current_status'].value_counts()

1    993
0    243
Name: current_status, dtype: int64

In [ ]:
original_rounds_df[original_rounds_df['current_status'].isna()]

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team


In [ ]:
original_rounds_df[original_rounds_df['foundation_year'].isna()]

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team


In [ ]:
# data types
original_rounds_df.dtypes

id_company                     int64
category                      object
deal_type                     object
foundation_year                int64
number_employees             float64
increase_employees             int64
number_investors               int64
current_status                 int64
foreign_vc                     int64
local_vc                       int64
round_year                     int64
round_amount                 float64
support_company              float64
manufacture_equipment        float64
develop_new_tech             float64
grow_company                 float64
add_new_services             float64
develop_business             float64
financial_banking            float64
develop_products_services    float64
marketing_strategies         float64
expand_internationality      float64
consolidate_company          float64
increase_team                float64
dtype: object

In [ ]:
# Drop the null columns where all values are null
rounds_df = original_rounds_df.dropna(axis='columns', how='all')

# Drop the null rows
# rounds_df = rounds_df.dropna()
rounds_df.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team
0,3,E-Commerce,Seed,2017,33.0,8,3,1,0,1,2017,1400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,SaaS,Seed,2018,10.0,0,2,0,0,1,2018,10500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,SaaS,Angel,2015,11.0,14,1,1,0,1,2015,200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,IT services,Seed,2016,10.0,0,1,0,0,1,2016,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23,E-Commerce,Seed,2013,1001.0,0,1,1,1,0,2013,150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Dataframe length
len(rounds_df)

1236

In [ ]:
# Create Rounds Encode
db_rounds_Encode = rounds_df.copy()

In [ ]:
# Define deal_type Encoding Function (for ordering purposes)

def fnc_encode_deal_type(str_deal_type):
  if (str_deal_type == 'Convertible Note / Debt'):
    return 0
  elif (str_deal_type == 'Angel'):
    return 1
  elif (str_deal_type == 'Accelerator/Incubator'):
    return 2
  elif (str_deal_type == 'Seed'):
    return 3
  elif (str_deal_type == 'Early Stage VC'):
    return 4
  elif (str_deal_type == 'Series A'):
    return 5
  elif (str_deal_type == 'Series B'):
    return 6
  elif (str_deal_type == 'Series C'):
    return 7
  elif (str_deal_type == 'Series D'):
    return 8
  elif (str_deal_type == 'Series E'):
    return 9
  elif (str_deal_type == 'Venture - Series Unknown'):
    return 0
  else:
    return 0

In [ ]:
# Create id_deal_type
db_rounds_Encode['id_deal_type'] = db_rounds_Encode[['deal_type']].applymap(lambda x : fnc_encode_deal_type(x))

In [ ]:
# Encoding labels
le = LabelEncoder()
db_rounds_Encode['deal_type'] = le.fit_transform(db_rounds_Encode['deal_type'])

# Get the name of the columns
columnsName = db_rounds_Encode.columns.tolist()
#columnsName.remove('rounds_funding_factor')

# convert variables to numerical
for column in columnsName:
  db_rounds_Encode[column] = pd.to_numeric(db_rounds_Encode[column], errors='coerce').fillna(0).astype(np.int64)

db_rounds_Encode.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
# data types
db_rounds_Encode.dtypes

id_company                   int64
category                     int64
deal_type                    int64
foundation_year              int64
number_employees             int64
increase_employees           int64
number_investors             int64
current_status               int64
foreign_vc                   int64
local_vc                     int64
round_year                   int64
round_amount                 int64
support_company              int64
manufacture_equipment        int64
develop_new_tech             int64
grow_company                 int64
add_new_services             int64
develop_business             int64
financial_banking            int64
develop_products_services    int64
marketing_strategies         int64
expand_internationality      int64
consolidate_company          int64
increase_team                int64
id_deal_type                 int64
dtype: object

In [ ]:
len(db_rounds_Encode)

1236

# Rounds Preprocessing, 1st Stage (Current Info / Success)


- Obtain **Basic Companies Info** (Current Info) from Rounds Table as:
  - Id, category, foundation year, current status, current number of employees, current number of investors
  - Seed round (year and amount)
  - Last round (year and amount)
  - Years since last funding
  - Current number of employees
  - Current number of investors
  - Average years per round
  - [Funding allocation?]
  - [foregin vc, local vc?]
- **Define success** in terms of:
  - Had a funding round of more than $20 Million USD (Soonicorn)
  - Having 50 or more employees
  - Having 100 or more employees

In [ ]:
# Data base of VC Rounds in entrepreneurs 
df_rounds = db_rounds_Encode

In [ ]:
df_rounds.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
# Ensure rounds order by id_company, round_year, id_deal_type
df_rounds.sort_values(['id_company','round_year','id_deal_type'],inplace=True)
df_rounds.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
# Rest Index
df_rounds.reset_index(drop=True,inplace=True)
df_rounds.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
# Add id_round equal to index
df_rounds['id_round']=df_rounds.index
df_rounds.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3,4


In [ ]:
# Check types
df_rounds.dtypes

id_company                   int64
category                     int64
deal_type                    int64
foundation_year              int64
number_employees             int64
increase_employees           int64
number_investors             int64
current_status               int64
foreign_vc                   int64
local_vc                     int64
round_year                   int64
round_amount                 int64
support_company              int64
manufacture_equipment        int64
develop_new_tech             int64
grow_company                 int64
add_new_services             int64
develop_business             int64
financial_banking            int64
develop_products_services    int64
marketing_strategies         int64
expand_internationality      int64
consolidate_company          int64
increase_team                int64
id_deal_type                 int64
id_round                     int64
dtype: object

In [ ]:
# Convert object to numeric
# df_rounds['round_amount']=pd.to_numeric(df_rounds['round_amount'], errors='coerce').astype('Int64').fillna(0)

In [ ]:
# Recheck data types
df_rounds.dtypes

id_company                   int64
category                     int64
deal_type                    int64
foundation_year              int64
number_employees             int64
increase_employees           int64
number_investors             int64
current_status               int64
foreign_vc                   int64
local_vc                     int64
round_year                   int64
round_amount                 int64
support_company              int64
manufacture_equipment        int64
develop_new_tech             int64
grow_company                 int64
add_new_services             int64
develop_business             int64
financial_banking            int64
develop_products_services    int64
marketing_strategies         int64
expand_internationality      int64
consolidate_company          int64
increase_team                int64
id_deal_type                 int64
id_round                     int64
dtype: object

In [ ]:
df_rounds[['round_year']].isna().sum()

round_year    0
dtype: int64

In [ ]:
df_rounds[['round_year']].value_counts()

round_year
2016          184
2017          180
2015          141
2018          126
2019          121
2014          118
2020          102
2013          100
2021           89
2012           41
2011           19
2010            5
2008            5
2009            2
2006            1
2004            1
2000            1
dtype: int64

In [ ]:
df_rounds

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,4429,0,10,2021,259,249,0,1,0,0,2021,3100000,0,0,0,0,0,0,0,0,0,0,0,0,3,1231
1232,4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
1233,4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233
1234,4435,0,0,2021,9,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1234


In [ ]:
len(df_rounds)

1236

In [ ]:
# Group and Count
grouped_df_rounds_count = df_rounds.groupby(by=['id_company']).count()
grouped_df_rounds_count

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
13,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
23,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
4433,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4434,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Group and get all min by id_company
grouped_df_rounds_min = df_rounds.groupby(by=['id_company']).min(numeric_only=True)
grouped_df_rounds_min

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,10,2013,1001,0,0,1,0,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,0,2018,259,249,0,1,0,0,2018,750000,0,0,0,0,0,0,0,0,0,0,0,0,2,1228
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


In [ ]:
# Group and obtain all max by id_company
grouped_df_rounds_max = df_rounds.groupby(by=['id_company']).max(numeric_only=True)
grouped_df_rounds_max

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,12,2021,1001,0,4,1,1,0,2021,40000000,0,0,0,1,0,0,0,0,0,1,0,0,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,11,2021,259,249,9,1,1,1,2021,4960000,0,0,0,1,0,0,0,0,0,0,0,0,5,1231
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


In [ ]:
# Get first rounds array
first_rounds = []
for round in grouped_df_rounds_min[['id_round']].to_numpy():
  first_rounds.append(round[0])

# first_rounds

In [ ]:
# Create first rounds data frame
df_first_rounds = df_rounds.loc[first_rounds]

# Redifine first rounds index to id_company
df_first_rounds.set_index('id_company',inplace=True)

df_first_rounds

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,0,2018,259,249,9,1,1,1,2018,750000,0,0,0,1,0,0,0,0,0,0,0,0,2,1228
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


In [ ]:
# Get last rounds array
last_rounds = []
for round in grouped_df_rounds_max[['id_round']].to_numpy():
  last_rounds.append(round[0])

# last_rounds

In [ ]:
# Create last rounds data frame
df_last_rounds = df_rounds.loc[last_rounds]

# Redifine last rounds index to id_company
df_last_rounds.set_index('id_company',inplace=True)

df_last_rounds

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,12,2021,1001,0,2,1,1,0,2021,40000000,0,0,0,1,0,0,0,0,0,1,0,0,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,10,2021,259,249,0,1,0,0,2021,3100000,0,0,0,0,0,0,0,0,0,0,0,0,3,1231
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


# Create Companies Data Frame with Rounds 1st Stage

In [ ]:
# Create Dataframe

df_companies = df_first_rounds[['foundation_year']]

df_companies['category'] = df_first_rounds['category']

df_companies['seed_year']=df_first_rounds['round_year']

df_companies['seed_amount']=df_first_rounds['round_amount']

df_companies['last_funding_year']=df_last_rounds['round_year']

df_companies['amount_last_funding']=df_last_rounds['round_amount']

df_companies['month_since_last_funding']=(2021-df_companies['last_funding_year'])*12

df_companies[['number_employees','increase_employees','number_investors']] = grouped_df_rounds_min[['number_employees','increase_employees','number_investors']]

df_companies['current_status']=df_first_rounds['current_status']



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [ ]:
df_companies['rounds_funding_factor'] = (2021-df_companies['last_funding_year'])/grouped_df_rounds_count['category']

df_companies['soonicorn'] = grouped_df_rounds_max[['round_amount']].applymap(lambda x: 1 if ( x >20000000 ) else 0  )

df_companies['more50_employees'] = grouped_df_rounds_max[['number_employees']].applymap(lambda x: 1 if ( x >=50 ) else 0  )

df_companies['more100_employees'] = grouped_df_rounds_max[['number_employees']].applymap(lambda x: 1 if ( x >=100 ) else 0  )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [ ]:
#Show current companies Data Frame
df_companies

,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees
id_company,,,,,,,,,,,,,,,
3,2017,0,2017,1400000,2017,1400000,48,33,8,3,1,4.0,0,0,0
7,2018,0,2018,10500,2018,10500,36,10,0,2,0,3.0,0,0,0
10,2015,0,2015,200000,2015,200000,72,11,14,1,1,6.0,0,0,0
13,2016,0,2016,50000,2016,50000,60,10,0,1,0,5.0,0,0,0
23,2013,0,2013,150000,2021,40000000,0,1001,0,0,1,0.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,2018,0,2018,750000,2021,3100000,0,259,249,0,1,0.0,0,1,1
4433,2021,0,2021,225000000,2021,225000000,0,125,124,2,1,0.0,1,1,1
4434,2021,0,2021,130000,2021,130000,0,11,0,1,1,0.0,0,0,0


# Rounds 2nd Stage (Data at Decision Point)


- The **Decision Point (dp)** is the moment when the VC usually is considering to invest in a new company that is **before a C Series Round**
- Therefore the Rounds Data Frame is filtered to obtain only the rounds Before the decision pont
- From Rounds (on decision point)
  - Decision point Year
  - Rounds count
  - Sum of rounds ammount
  - Max amount

In [ ]:
# Filter id_deal_type under 7 (Series C)

df_rounds_filtered = df_rounds[df_rounds['id_deal_type']<7]
df_rounds_filtered.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3,4


In [ ]:
# Group and obtain count by id_company
grouped_df_rounds_filtered_count = df_rounds_filtered.groupby(by=['id_company']).count()
grouped_df_rounds_filtered_count

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
13,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
23,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
4433,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4434,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Group and obtain all min by id_company
grouped_df_rounds_filtered_min = df_rounds_filtered.groupby(by=['id_company']).min(numeric_only=True)
grouped_df_rounds_filtered_min

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,10,2013,1001,0,0,1,0,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,0,2018,259,249,0,1,0,0,2018,750000,0,0,0,0,0,0,0,0,0,0,0,0,2,1228
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


In [ ]:
# Group and obtain all max by id_company
grouped_df_rounds_filtered_max = df_rounds_filtered.groupby(by=['id_company']).max(numeric_only=True)
grouped_df_rounds_filtered_max

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,12,2021,1001,0,4,1,1,0,2021,40000000,0,0,0,1,0,0,0,0,0,1,0,0,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,11,2021,259,249,9,1,1,1,2021,4960000,0,0,0,1,0,0,0,0,0,0,0,0,5,1231
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


In [ ]:
grouped_df_rounds_filtered_max.isna().sum()

category                     0
deal_type                    0
foundation_year              0
number_employees             0
increase_employees           0
number_investors             0
current_status               0
foreign_vc                   0
local_vc                     0
round_year                   0
round_amount                 0
support_company              0
manufacture_equipment        0
develop_new_tech             0
grow_company                 0
add_new_services             0
develop_business             0
financial_banking            0
develop_products_services    0
marketing_strategies         0
expand_internationality      0
consolidate_company          0
increase_team                0
id_deal_type                 0
id_round                     0
dtype: int64

In [ ]:
# Group and obtain all sum by id_company
grouped_df_rounds_filtered_sum = df_rounds_filtered.groupby(by=['id_company']).sum(numeric_only=True)
grouped_df_rounds_filtered_sum

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,53,10094,5005,0,10,5,4,0,10094,54800000,0,0,0,1,0,0,0,0,0,1,0,0,20,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,31,8077,1036,996,24,4,3,1,8077,12510000,0,0,0,1,0,0,0,0,0,0,0,0,13,4918
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


In [ ]:
# Group and obtain all mean by id_company
grouped_df_rounds_filtered_mean = df_rounds_filtered.groupby(by=['id_company']).mean(numeric_only=True)
grouped_df_rounds_filtered_mean

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0.0,10.00,2017.00,33.0,8.0,3.0,1.0,0.00,1.00,2017.00,1400000.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,0.0
7,0.0,10.00,2018.00,10.0,0.0,2.0,0.0,0.00,1.00,2018.00,10500.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,1.0
10,0.0,1.00,2015.00,11.0,14.0,1.0,1.0,0.00,1.00,2015.00,200000.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,2.0
13,0.0,10.00,2016.00,10.0,0.0,1.0,0.0,0.00,1.00,2016.00,50000.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,3.0
23,0.0,10.60,2018.80,1001.0,0.0,2.0,1.0,0.80,0.00,2018.80,10960000.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,4.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0.0,7.75,2019.25,259.0,249.0,6.0,1.0,0.75,0.25,2019.25,3127500.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.25,1229.5
4433,0.0,12.00,2021.00,125.0,124.0,2.0,1.0,1.00,0.00,2021.00,225000000.0,1.0,0.0,1.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.00,1232.0
4434,0.0,0.00,2021.00,11.0,0.0,1.0,1.0,1.00,0.00,2021.00,130000.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,1233.0


In [ ]:
# Get last rounds array
dp_last_rounds = []
for round in grouped_df_rounds_filtered_max[['id_round']].to_numpy():
  dp_last_rounds.append(round[0])

# dp_last_rounds

In [ ]:


# Create last rounds data frame
df_dp_last_rounds = df_rounds.loc[dp_last_rounds]

# Redifine last rounds index to id_company
df_dp_last_rounds.set_index('id_company',inplace=True)

df_dp_last_rounds

,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team,id_deal_type,id_round
id_company,,,,,,,,,,,,,,,,,,,,,,,,,
3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0,3,0
7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0,3,1
10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0,1,2
13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0,3,3
23,0,12,2021,1001,0,2,1,1,0,2021,40000000,0,0,0,1,0,0,0,0,0,1,0,0,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0,10,2021,259,249,0,1,0,0,2021,3100000,0,0,0,0,0,0,0,0,0,0,0,0,3,1231
4433,0,12,2021,125,124,2,1,1,0,2021,225000000,1,0,1,1,0,0,0,0,0,0,0,0,6,1232
4434,0,0,2021,11,0,1,1,1,0,2021,130000,0,0,0,0,0,0,0,0,0,0,0,0,2,1233


# Integrate Info from Rounds 2nd stage in Companies Data Frame

In [ ]:
# dp_year
df_companies['dp_year'] = grouped_df_rounds_filtered_max['round_year']

# dp_rondas_count
df_companies['dp_rounds_count'] = grouped_df_rounds_filtered_count['category']

# dp_roundas_amount_sum
df_companies['dp_rounds_amount_sum'] = grouped_df_rounds_filtered_sum['round_amount']

# dp_roundas_amount_max
df_companies['dp_rounds_amount_max'] = grouped_df_rounds_filtered_max['round_amount']

# dp_last_round_year
df_companies['dp_last_round_year'] = df_dp_last_rounds['round_year']

# dp_last_round_amount
df_companies['dp_last_round_amount'] = df_dp_last_rounds['round_amount']

# show df_companies
df_companies.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount
id_company,,,,,,,,,,,,,,,,,,,,,
3,2017,0,2017,1400000,2017,1400000,48,33,8,3,1,4.0,0,0,0,2017.0,1.0,1400000.0,1400000.0,2017.0,1400000.0
7,2018,0,2018,10500,2018,10500,36,10,0,2,0,3.0,0,0,0,2018.0,1.0,10500.0,10500.0,2018.0,10500.0
10,2015,0,2015,200000,2015,200000,72,11,14,1,1,6.0,0,0,0,2015.0,1.0,200000.0,200000.0,2015.0,200000.0
13,2016,0,2016,50000,2016,50000,60,10,0,1,0,5.0,0,0,0,2016.0,1.0,50000.0,50000.0,2016.0,50000.0
23,2013,0,2013,150000,2021,40000000,0,1001,0,0,1,0.0,1,1,1,2021.0,5.0,54800000.0,40000000.0,2021.0,40000000.0


# Founders Table Data Cleaning

In [ ]:
# Open Conection

# Connexion throught psycopg2 Google Cloud SQL data base
conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

In [ ]:
# Query to connect with SQL data base from Founders Demographics 
SQL_Query = pd.read_sql_query('''select * FROM "Founders" ''', conn)
founders_df = pd.DataFrame(SQL_Query)

In [ ]:
# Close Connection
conn.close()

In [ ]:
#Count NaN
founders_df.isna().sum()

id_founder                              0
id_company                              0
general_location                       11
manager_exp                             0
clevel_exp                              0
engineering_exp                         0
sales_exp                               0
finance_exp                             0
human_capital_exp                       0
consulting_exp                          0
founder_exp                             1
foundation_year                         7
years_of_experience_as_entreprenuer     7
age_of_first_entrepreneurship          74
age                                     6
years_of_experience                     6
number_of_startups                      0
gender                                  0
foreing_studies                         0
foreing_work_experience                 0
foreing_founder                         0
dtype: int64

In [ ]:
# Data base of Founders Demographics  
founders_df.head(20)

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,"Ciudad de México, México",1,1,0,0,0,0,0,1.0,2020.0,12.0,31.0,32.0,13.0,4,M,0,0,0
1,2,23,"Ciudad de México, México",0,1,0,0,0,0,0,1.0,2014.0,4.0,25.0,32.0,11.0,2,M,0,0,0
2,3,141,México,0,1,0,0,0,0,0,0.0,2015.0,0.0,22.0,28.0,6.0,1,M,0,1,0
3,4,152,"Ciudad de México, México",0,0,0,0,1,0,1,1.0,2017.0,6.0,28.0,32.0,10.0,3,M,1,1,1
4,5,155,México,1,1,0,0,0,0,0,0.0,2015.0,10.0,27.0,33.0,16.0,1,M,1,0,0
5,6,167,None,0,1,0,0,0,0,0,1.0,2003.0,0.0,NaN,41.0,18.0,1,M,0,0,0
6,7,185,"Ciudad de México, México",0,0,0,0,0,0,0,1.0,2021.0,20.0,41.0,41.0,20.0,4,M,1,0,0
7,8,233,"Ciudad de México, México",1,1,0,0,0,0,1,1.0,2016.0,13.0,34.0,39.0,18.0,2,F,0,1,1
8,9,1799,None,1,1,1,0,0,0,0,1.0,2013.0,25.0,54.0,62.0,33.0,1,M,0,1,0
9,10,284,"Ciudad de México, México",0,0,0,0,1,0,0,1.0,2016.0,2.0,24.0,29.0,7.0,1,M,0,0,0


In [ ]:
# data types
founders_df.dtypes

id_founder                               int64
id_company                               int64
general_location                        object
manager_exp                              int64
clevel_exp                               int64
engineering_exp                          int64
sales_exp                                int64
finance_exp                              int64
human_capital_exp                        int64
consulting_exp                           int64
founder_exp                            float64
foundation_year                        float64
years_of_experience_as_entreprenuer    float64
age_of_first_entrepreneurship          float64
age                                    float64
years_of_experience                    float64
number_of_startups                       int64
gender                                  object
foreing_studies                          int64
foreing_work_experience                  int64
foreing_founder                          int64
dtype: object

In [ ]:
# Drop the null columns where all values are null
founders_df = founders_df.dropna(axis='columns', how='all')

# Drop the null rows
# founders_df = founders_df.dropna()
founders_df.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,"Ciudad de México, México",1,1,0,0,0,0,0,1.0,2020.0,12.0,31.0,32.0,13.0,4,M,0,0,0
1,2,23,"Ciudad de México, México",0,1,0,0,0,0,0,1.0,2014.0,4.0,25.0,32.0,11.0,2,M,0,0,0
2,3,141,México,0,1,0,0,0,0,0,0.0,2015.0,0.0,22.0,28.0,6.0,1,M,0,1,0
3,4,152,"Ciudad de México, México",0,0,0,0,1,0,1,1.0,2017.0,6.0,28.0,32.0,10.0,3,M,1,1,1
4,5,155,México,1,1,0,0,0,0,0,0.0,2015.0,10.0,27.0,33.0,16.0,1,M,1,0,0


In [ ]:
# Encoding labels
le = LabelEncoder()
db_founders_Encode = founders_df.copy()
db_founders_Encode['general_location'] = db_founders_Encode['general_location'].fillna("nan", inplace = True)
db_founders_Encode['general_location'] = le.fit_transform(db_founders_Encode['general_location'])
db_founders_Encode['gender'] = le.fit_transform(db_founders_Encode['gender'])
#db_founders_Encode['category'] = le.fit_transform(db_founders_Encode['category'])

# Get the name of the columns
columnsName = db_founders_Encode.columns.tolist()

# convert variables to numerical
for column in columnsName:
  db_founders_Encode[column] = pd.to_numeric(db_founders_Encode[column], errors='coerce').fillna(0).astype(np.int64)

db_founders_Encode.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,0,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0
1,2,23,0,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0
2,3,141,0,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0
3,4,152,0,0,0,0,0,1,0,1,1,2017,6,28,32,10,3,1,1,1,1
4,5,155,0,1,1,0,0,0,0,0,0,2015,10,27,33,16,1,1,1,0,0


In [ ]:
# data types
db_founders_Encode.dtypes

id_founder                             int64
id_company                             int64
general_location                       int64
manager_exp                            int64
clevel_exp                             int64
engineering_exp                        int64
sales_exp                              int64
finance_exp                            int64
human_capital_exp                      int64
consulting_exp                         int64
founder_exp                            int64
foundation_year                        int64
years_of_experience_as_entreprenuer    int64
age_of_first_entrepreneurship          int64
age                                    int64
years_of_experience                    int64
number_of_startups                     int64
gender                                 int64
foreing_studies                        int64
foreing_work_experience                int64
foreing_founder                        int64
dtype: object


# Founders preprocessing (Calculate founders data at decision point (dp))

- Example:
  - Years of experience at decision pont
    - dp_years_of_experience = dp_Year - (2021 - current_years_of_experience) 

- Obtain max, sum and mean of all variables

In [ ]:
# Add Company decision point year
df_founders = db_founders_Encode.join(df_companies[['soonicorn','more50_employees','more100_employees','dp_year']],on='id_company',how='inner')

In [ ]:
df_founders

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder,soonicorn,more50_employees,more100_employees,dp_year
0,1,7,0,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0,0,0,0,2018.0
1,2,23,0,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0,1,1,1,2021.0
2,3,141,0,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0,0,1,1,2021.0
193,194,141,0,0,0,1,0,0,0,0,1,2015,7,25,31,13,2,1,0,0,0,0,1,1,2021.0
248,249,141,0,0,1,0,0,0,0,0,1,2015,18,47,53,24,5,1,0,1,1,0,1,1,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,634,2336,0,1,1,0,1,0,0,0,1,2020,11,25,26,12,1,0,1,0,0,0,0,0,2015.0
634,635,3211,0,0,1,1,0,0,0,0,0,2016,5,30,35,10,2,1,1,1,0,0,0,0,2018.0
635,636,1582,0,1,1,0,0,0,0,0,0,2014,7,29,35,6,1,1,1,1,1,1,1,1,2020.0
636,637,1582,0,1,0,0,1,1,0,0,0,2014,7,30,37,7,1,1,1,1,1,1,1,1,2020.0


In [ ]:
# Decision Point Year values
df_founders['dp_year'].value_counts()

2017.0    88
2021.0    84
2018.0    84
2019.0    81
2020.0    80
2016.0    75
2015.0    55
2014.0    51
2013.0    19
2012.0    11
2011.0     5
2008.0     3
2000.0     1
Name: dp_year, dtype: int64

In [ ]:
df_founders.notna().sum()

id_founder                             638
id_company                             638
general_location                       638
manager_exp                            638
clevel_exp                             638
engineering_exp                        638
sales_exp                              638
finance_exp                            638
human_capital_exp                      638
consulting_exp                         638
founder_exp                            638
foundation_year                        638
years_of_experience_as_entreprenuer    638
age_of_first_entrepreneurship          638
age                                    638
years_of_experience                    638
number_of_startups                     638
gender                                 638
foreing_studies                        638
foreing_work_experience                638
foreing_founder                        638
soonicorn                              638
more50_employees                       638
more100_emp

In [ ]:
# Define Function
def fnc_dp_years_of_experience(dp_year,first_employment_year):
  if ( ( dp_year > 0 ) and ( first_employment_year > 0 )  and ( dp_year >= first_employment_year ) ):
    return (dp_year - first_employment_year)
  else:
    return 0


In [ ]:
# First Employment Year
df_founders['first_employment_year'] = 2021-df_founders['years_of_experience']

# Decision point years of experience
df_founders['dp_years_of_experience'] = df_founders.apply(lambda x : fnc_dp_years_of_experience(x['dp_year'],x['first_employment_year']),axis=1)


In [ ]:
# first_employment_year values
df_founders['first_employment_year'].value_counts()

2011    65
2007    51
2010    50
2008    43
2009    39
2006    38
2012    37
2005    35
2013    34
2014    27
2003    25
2015    22
2004    21
2000    18
2001    16
2016    16
1999    16
1996     9
2002     9
2019     8
1997     7
2021     6
2017     6
2018     5
1995     5
1994     5
1990     3
1992     3
1998     3
1989     3
1993     2
1991     2
1988     2
1985     1
1979     1
1982     1
2020     1
1986     1
1987     1
1969     1
Name: first_employment_year, dtype: int64

In [ ]:
# dp_years_of_experience values
df_founders['dp_years_of_experience'].value_counts()

9.0     54
7.0     49
8.0     48
6.0     42
10.0    40
5.0     38
13.0    36
12.0    36
11.0    35
4.0     35
14.0    27
2.0     22
16.0    21
3.0     20
1.0     18
0.0     18
15.0    16
17.0    16
18.0    12
19.0    10
20.0     7
21.0     6
23.0     5
24.0     5
22.0     4
32.0     4
27.0     3
25.0     3
33.0     2
26.0     2
28.0     2
43.0     1
29.0     1
Name: dp_years_of_experience, dtype: int64

In [ ]:
df_founders.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder,soonicorn,more50_employees,more100_employees,dp_year,first_employment_year,dp_years_of_experience
0,1,7,0,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0,0,0,0,2018.0,2008,10.0
1,2,23,0,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0,1,1,1,2021.0,2010,11.0
2,3,141,0,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0,0,1,1,2021.0,2015,6.0
193,194,141,0,0,0,1,0,0,0,0,1,2015,7,25,31,13,2,1,0,0,0,0,1,1,2021.0,2008,13.0
248,249,141,0,0,1,0,0,0,0,0,1,2015,18,47,53,24,5,1,0,1,1,0,1,1,2021.0,1997,24.0


In [ ]:
# Define Function
def fnc_dp_years_of_experience_as_entreprenuer(dp_year,first_first_etreprenuership_year):
  if ( (dp_year > 0) and (first_etreprenuership_year >0) and (dp_year >= first_etreprenuership_year) ) :
    return (dp_year - first_etreprenuership_year)
  else:
    return 0

In [ ]:
# First Entreprenuership year
df_founders['first_etreprenuership_year'] = 2021-df_founders['years_of_experience_as_entreprenuer'] # Ojo traer de la base de datos

# Decision point point years of experience as entreprenuer
df_founders['dp_years_of_experience_as_entreprenuer'] = df_founders.apply(lambda x : fnc_dp_years_of_experience(x['dp_year'],x['first_etreprenuership_year']),axis=1)

In [ ]:
df_founders.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder,soonicorn,more50_employees,more100_employees,dp_year,first_employment_year,dp_years_of_experience,first_etreprenuership_year,dp_years_of_experience_as_entreprenuer
0,1,7,0,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0,0,0,0,2018.0,2008,10.0,2009,9.0
1,2,23,0,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0,1,1,1,2021.0,2010,11.0,2017,4.0
2,3,141,0,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0,0,1,1,2021.0,2015,6.0,2021,0.0
193,194,141,0,0,0,1,0,0,0,0,1,2015,7,25,31,13,2,1,0,0,0,0,1,1,2021.0,2008,13.0,2014,7.0
248,249,141,0,0,1,0,0,0,0,0,1,2015,18,47,53,24,5,1,0,1,1,0,1,1,2021.0,1997,24.0,2003,18.0


In [ ]:
founders_selected_columns = [
  'id_company',
  'manager_exp',
  'clevel_exp',
  'engineering_exp',
  'sales_exp',
  'finance_exp',
  'human_capital_exp',
  #'marketing_exp',
  'consulting_exp',
  'founder_exp',
  #'abroad_experience',
  #'worked_on_plus100_company',
  #'software_exp',
  #'insurance_exp',
  #'agriculture_related_deg',
  #'business_deg',
  #'entreprenueurship_deg',
  #'financial_deg',
  #'marketing_deg',
  #'maths_engineering_deg',
  #'project_management',
  #'research',
  #'sales',
  #'soft_skills',
  #'software_skills',
  # foundation_year
  # years_of_experience_as_entreprenuer
  # age_of_first_entrepreneurship
  # age
  #	years_of_experience	
  'number_of_startups',
  'gender',
  #	category
  'foreing_studies',
  'foreing_work_experience',
  'foreing_founder',
  # soonicorn	
  # more50_employees	
  # more100_employees	
  # dp_year	
  # first_employment_year	
  'dp_years_of_experience',
  # 'first_etreprenuership_year'
  'dp_years_of_experience_as_entreprenuer',

]

founders_selected_columns

['id_company',
 'manager_exp',
 'clevel_exp',
 'engineering_exp',
 'sales_exp',
 'finance_exp',
 'human_capital_exp',
 'consulting_exp',
 'founder_exp',
 'number_of_startups',
 'gender',
 'foreing_studies',
 'foreing_work_experience',
 'foreing_founder',
 'dp_years_of_experience',
 'dp_years_of_experience_as_entreprenuer']

In [ ]:
df_founders_selected_columns = df_founders[founders_selected_columns]

In [ ]:
# define rename function
def fnc_rename_columns(in_prefix, in_df):
  list_columns = in_df.columns.to_list()
  i = 0
  for column in list_columns:
    in_df.rename(columns={list_columns[i] : f"{in_prefix}_{list_columns[i]}"},inplace=True)
    i = i + 1
  # print(list_columns) 
  return in_df

In [ ]:
# Group and obtain all max by id_company
grouped_df_founders_max = df_founders_selected_columns.groupby(by=['id_company']).max(numeric_only=True)
grouped_df_founders_max = fnc_rename_columns("max",grouped_df_founders_max)
grouped_df_founders_max

,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer
id_company,,,,,,,,,,,,,,,
3,1,1,0,0,0,0,0,1,3,1,1,0,0,12.0,8.0
7,1,1,0,0,0,0,0,1,4,1,0,0,0,10.0,9.0
10,1,1,0,0,0,0,0,1,1,1,0,0,0,12.0,2.0
23,0,1,0,0,0,0,0,1,2,1,0,0,0,11.0,4.0
51,0,1,1,0,0,0,0,0,1,1,1,0,0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,1,1,0,1,0,0,1,1,1,1,1,1,0,12.0,11.0
4433,0,1,0,0,1,0,1,1,1,1,1,0,1,9.0,8.0
4434,0,0,1,0,0,0,0,1,2,1,1,0,0,6.0,4.0


In [ ]:
# Group and obtain sum by id_company
grouped_df_founders_sum = df_founders_selected_columns.groupby(by=['id_company']).sum(numeric_only=True)
grouped_df_founders_sum = fnc_rename_columns("sum",grouped_df_founders_sum)
grouped_df_founders_sum

,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer
id_company,,,,,,,,,,,,,,,
3,1,1,0,0,0,0,0,2,4,2,1,0,0,22.0,15.0
7,1,1,0,0,0,0,0,1,4,1,0,0,0,10.0,9.0
10,1,1,0,0,0,0,0,1,1,1,0,0,0,12.0,2.0
23,0,1,0,0,0,0,0,1,2,1,0,0,0,11.0,4.0
51,0,1,1,0,0,0,0,0,1,1,1,0,0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,1,1,0,1,0,0,1,1,1,1,1,1,0,12.0,11.0
4433,0,1,0,0,1,0,1,1,1,1,1,0,1,9.0,8.0
4434,0,0,1,0,0,0,0,1,2,1,1,0,0,6.0,4.0


In [ ]:
# Group and obtain mean by id_company
grouped_df_founders_mean = df_founders_selected_columns.groupby(by=['id_company']).mean(numeric_only=True)
grouped_df_founders_mean = fnc_rename_columns("mean",grouped_df_founders_mean)
grouped_df_founders_mean

,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
id_company,,,,,,,,,,,,,,,
3,0.5,0.5,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.5,0.0,0.0,11.0,7.5
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
10,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
23,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
51,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0
4433,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0
4434,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,6.0,4.0


# Founders Check Values

In [ ]:
df_founders['foundation_year'].value_counts()

2015    91
2016    73
2014    68
2019    64
2018    59
2017    59
2013    53
2020    48
2012    41
2021    20
2011    20
2010    10
0        7
2009     5
2008     4
2007     3
2003     3
2002     3
2001     2
1999     2
2006     1
2004     1
1997     1
Name: foundation_year, dtype: int64

In [ ]:
df_founders[df_founders['dp_years_of_experience']<0]

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder,soonicorn,more50_employees,more100_employees,dp_year,first_employment_year,dp_years_of_experience,first_etreprenuership_year,dp_years_of_experience_as_entreprenuer


In [ ]:
df_founders['dp_years_of_experience'].value_counts()

9.0     54
7.0     49
8.0     48
6.0     42
10.0    40
5.0     38
13.0    36
12.0    36
11.0    35
4.0     35
14.0    27
2.0     22
16.0    21
3.0     20
1.0     18
0.0     18
15.0    16
17.0    16
18.0    12
19.0    10
20.0     7
21.0     6
23.0     5
24.0     5
22.0     4
32.0     4
27.0     3
25.0     3
33.0     2
26.0     2
28.0     2
43.0     1
29.0     1
Name: dp_years_of_experience, dtype: int64

# Integrate Info from Founders to Companies Data Frame

In [ ]:
# Add max
df_companies_with_rounds_and_founders = df_companies.join(grouped_df_founders_max,how='inner')

# Add sum
df_companies_with_rounds_and_founders = df_companies_with_rounds_and_founders.join(grouped_df_founders_sum,how='inner')

# Add mean
df_companies_with_rounds_and_founders = df_companies_with_rounds_and_founders.join(grouped_df_founders_mean,how='inner')

In [ ]:
df_companies_with_rounds_and_founders.head()

,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
id_company,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,2017,0,2017,1400000,2017,1400000,48,33,8,3,1,4.0,0,0,0,2017.0,1.0,1400000.0,1400000.0,2017.0,1400000.0,1,1,0,0,0,0,0,1,3,1,1,0,0,12.0,8.0,1,1,0,0,0,0,0,2,4,2,1,0,0,22.0,15.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.5,0.0,0.0,11.0,7.5
7,2018,0,2018,10500,2018,10500,36,10,0,2,0,3.0,0,0,0,2018.0,1.0,10500.0,10500.0,2018.0,10500.0,1,1,0,0,0,0,0,1,4,1,0,0,0,10.0,9.0,1,1,0,0,0,0,0,1,4,1,0,0,0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
10,2015,0,2015,200000,2015,200000,72,11,14,1,1,6.0,0,0,0,2015.0,1.0,200000.0,200000.0,2015.0,200000.0,1,1,0,0,0,0,0,1,1,1,0,0,0,12.0,2.0,1,1,0,0,0,0,0,1,1,1,0,0,0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
23,2013,0,2013,150000,2021,40000000,0,1001,0,0,1,0.0,1,1,1,2021.0,5.0,54800000.0,40000000.0,2021.0,40000000.0,0,1,0,0,0,0,0,1,2,1,0,0,0,11.0,4.0,0,1,0,0,0,0,0,1,2,1,0,0,0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
51,2018,0,2018,20000,2018,20000,36,10,0,1,0,3.0,0,0,0,2018.0,1.0,20000.0,20000.0,2018.0,20000.0,0,1,1,0,0,0,0,0,1,1,1,0,0,5.0,0.0,0,1,1,0,0,0,0,0,1,1,1,0,0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0


In [ ]:
len(df_companies_with_rounds_and_founders)

510

In [ ]:
# Print all columns and data-types
i=0
for my_column in df_companies_with_rounds_and_founders.columns.to_list():
  print(f"{my_column}:        {df_companies_with_rounds_and_founders.dtypes[i]}")
  i = i + 1

foundation_year:        int64
category:        int64
seed_year:        int64
seed_amount:        int64
last_funding_year:        int64
amount_last_funding:        int64
month_since_last_funding:        int64
number_employees:        int64
increase_employees:        int64
number_investors:        int64
current_status:        int64
rounds_funding_factor:        float64
soonicorn:        int64
more50_employees:        int64
more100_employees:        int64
dp_year:        float64
dp_rounds_count:        float64
dp_rounds_amount_sum:        float64
dp_rounds_amount_max:        float64
dp_last_round_year:        float64
dp_last_round_amount:        float64
max_manager_exp:        int64
max_clevel_exp:        int64
max_engineering_exp:        int64
max_sales_exp:        int64
max_finance_exp:        int64
max_human_capital_exp:        int64
max_consulting_exp:        int64
max_founder_exp:        int64
max_number_of_startups:        int64
max_gender:        int64
max_foreing_studies:        i

In [ ]:
# Get the name of the columns
columnsName = df_companies_with_rounds_and_founders.columns.tolist()

# convert variables to numerical
for column in columnsName:
  df_companies_with_rounds_and_founders[column] = pd.to_numeric(df_companies_with_rounds_and_founders[column], errors='coerce').fillna(0).astype(np.int64)

db_founders_Encode.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,0,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0
1,2,23,0,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0
2,3,141,0,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0
3,4,152,0,0,0,0,0,1,0,1,1,2017,6,28,32,10,3,1,1,1,1
4,5,155,0,1,1,0,0,0,0,0,0,2015,10,27,33,16,1,1,1,0,0


# Create Table in PostgreSQL

In [ ]:
# Define table Name
Companies_table_name = "companies"

In [ ]:
df_companies_to_sql=df_companies_with_rounds_and_founders.reset_index()

In [ ]:
# df_companies_to_sql

In [ ]:
# Columns and Data-types
i=0
cols_and_types = ""
for my_column in df_companies_to_sql.columns.to_list():
  my_data_type = df_companies_to_sql.dtypes[i]
  
  cols_and_types = cols_and_types + f"{my_column} decimal,"

  i = i + 1

cols_and_types=cols_and_types[0:(len(cols_and_types)-1)]

cols_and_types=f"({cols_and_types})"

cols_and_types

'(id_company decimal,foundation_year decimal,category decimal,seed_year decimal,seed_amount decimal,last_funding_year decimal,amount_last_funding decimal,month_since_last_funding decimal,number_employees decimal,increase_employees decimal,number_investors decimal,current_status decimal,rounds_funding_factor decimal,soonicorn decimal,more50_employees decimal,more100_employees decimal,dp_year decimal,dp_rounds_count decimal,dp_rounds_amount_sum decimal,dp_rounds_amount_max decimal,dp_last_round_year decimal,dp_last_round_amount decimal,max_manager_exp decimal,max_clevel_exp decimal,max_engineering_exp decimal,max_sales_exp decimal,max_finance_exp decimal,max_human_capital_exp decimal,max_consulting_exp decimal,max_founder_exp decimal,max_number_of_startups decimal,max_gender decimal,max_foreing_studies decimal,max_foreing_work_experience decimal,max_foreing_founder decimal,max_dp_years_of_experience decimal,max_dp_years_of_experience_as_entreprenuer decimal,sum_manager_exp decimal,sum_cl

In [ ]:
# Open Conection

# Connexion throught psycopg2 Google Cloud SQL data base
conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

conn.set_isolation_level('ISOLATION_LEVEL_AUTOCOMMIT');

 
# Get cursor object from the database connection
psqlCursor = conn.cursor()
name_Table =  Companies_table_name
 
# Form the SQL statement - DROP TABLE
dropTableStmt = "DROP TABLE %s;"%name_Table;

# Execute the drop table command
psqlCursor.execute(dropTableStmt);

 

In [ ]:
# Free the resources

psqlCursor.close();

conn.close();

In [ ]:
# Query to connect with SQL data base from Founders Demographics 
SQL_Query = pd.read_sql_query('''DROP TABLE "companies2" ;''', conn)
drop_table_df = pd.DataFrame(SQL_Query)
drop_table_df

TypeError: ignored

In [ ]:
# Open Conection

# Connexion throught psycopg2 Google Cloud SQL data base
conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

In [ ]:

# Get cursor object from the database connection
psqlCursor = conn.cursor()
name_Table = Companies_table_name

 # Create table statement
sqlCreateTable = f"create table {name_Table} {cols_and_types};"

 # Create a table in PostgreSQL database
psqlCursor.execute(sqlCreateTable)
conn.commit()

 



ProgrammingError: ignored

In [ ]:
# Free the resources

psqlCursor.close();

conn.close();

In [ ]:
# Get the updated list of tables

# Open Conection

# Connexion throught psycopg2 Google Cloud SQL data base
conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

# Get cursor object from the database connection
psqlCursor = conn.cursor()

# sqlGetTableList = "SELECT table_schema,table_name FROM information_schema.tables where table_schema='test' ORDER BY table_schema,table_name ;"

sqlGetTableList = "SELECT table_schema,table_name FROM information_schema.tables WHERE table_schema='public' ORDER BY table_schema,table_name ;"

# sqlGetTableList = "\dt"
 

# Retrieve all the rows from the cursor
psqlCursor.execute(sqlGetTableList)
tables = psqlCursor.fetchall()


# Print the names of the tables

for table in tables:

    print(table)

('public', 'Founders')
('public', 'Rounds')
('public', 'companies')
('public', 'companies2')
('public', 'join_databases')


In [ ]:
# Query to connect with SQL data base
# SQL_stmt = ''' select * FROM ''' Companies_table_name &  """  '"
SQL_Query = pd.read_sql_query(''' select * FROM "''' + Companies_table_name + '''" ''', conn)
# SQL_Query = pd.read_sql_query(SQL_stmt, conn)
original_companies_df = pd.DataFrame(SQL_Query)
original_companies_df

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer


In [ ]:
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    for values in tuples:
      query  = f"INSERT INTO {table}({cols}) VALUES{values}"
      print(query)
      cursor = conn.cursor()
      
      try:
          cursor.execute(query)
          conn.commit()
      except (Exception, psycopg2.DatabaseError) as error:
          print("Error: %s" % error)
          conn.rollback()
          cursor.close()
          return 1
      print("execute_many() done")
      cursor.close()

In [ ]:
# Insert data on Company Table
execute_many(conn,df_companies_to_sql, Companies_table_name)

In [ ]:
# Query to connect with SQL data base
SQL_Query = pd.read_sql_query(''' select * FROM "''' + Companies_table_name + '''" ''', conn)
original_companies_df = pd.DataFrame(SQL_Query)
original_companies_df

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3.0,2017.0,0.0,2017.0,1400000.0,2017.0,1400000.0,48.0,33.0,8.0,3.0,1.0,4.0,0.0,0.0,0.0,2017.0,1.0,1400000.0,1400000.0,2017.0,1400000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,12.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,0.0,22.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7.0,2018.0,0.0,2018.0,10500.0,2018.0,10500.0,36.0,10.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,10500.0,10500.0,2018.0,10500.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10.0,2015.0,0.0,2015.0,200000.0,2015.0,200000.0,72.0,11.0,14.0,1.0,1.0,6.0,0.0,0.0,0.0,2015.0,1.0,200000.0,200000.0,2015.0,200000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23.0,2013.0,0.0,2013.0,150000.0,2021.0,40000000.0,0.0,1001.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2021.0,5.0,54800000.0,40000000.0,2021.0,40000000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51.0,2018.0,0.0,2018.0,20000.0,2018.0,20000.0,36.0,10.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,20000.0,20000.0,2018.0,20000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4429.0,2018.0,0.0,2018.0,750000.0,2021.0,3100000.0,0.0,259.0,249.0,0.0,1.0,0.0,0.0,1.0,1.0,2021.0,4.0,12510000.0,4960000.0,2021.0,3100000.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0
506,4433.0,2021.0,0.0,2021.0,225000000.0,2021.0,225000000.0,0.0,125.0,124.0,2.0,1.0,0.0,1.0,1.0,1.0,2021.0,1.0,225000000.0,225000000.0,2021.0,225000000.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0
507,4434.0,2021.0,0.0,2021.0,130000.0,2021.0,130000.0,0.0,11.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2021.0,1.0,130000.0,130000.0,2021.0,130000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0

In [ ]:
# Close Connection
conn.close()